In [1]:
from openff.toolkit.typing.engines.smirnoff.forcefield import get_available_force_fields

In [2]:
get_available_force_fields()

/scratch/users/yuzhuang/miniforge3_0507/envs/batter/lib/python3.12/site-packages/smirnoff99frosst/smirnoff99frosst.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


['openff_unconstrained-1.0.0.offxml',
 'tip4p_ew.offxml',
 'openff_unconstrained-2.0.0-rc.1.offxml',
 'openff-1.0.0-RC1.offxml',
 'tip3p-1.0.1.offxml',
 'tip3p.offxml',
 'opc-1.0.2.offxml',
 'openff_unconstrained-1.0.1.offxml',
 'openff_unconstrained-1.3.1-alpha.1.offxml',
 'spce-1.0.0.offxml',
 'openff-2.0.0.offxml',
 'tip4p_fb-1.0.0.offxml',
 'tip5p-1.0.0.offxml',
 'openff_unconstrained-2.0.0-rc.2.offxml',
 'openff-1.0.1.offxml',
 'openff-2.2.0.offxml',
 'opc3-1.0.1.offxml',
 'openff_unconstrained-1.1.1.offxml',
 'openff_unconstrained-2.2.1-rc1.offxml',
 'tip4p_ew-1.0.0.offxml',
 'openff_unconstrained-2.1.0-rc.1.offxml',
 'openff_unconstrained-1.2.1.offxml',
 'openff-2.2.1.offxml',
 'tip3p-1.0.0.offxml',
 'opc-1.0.1.offxml',
 'tip3p_fb.offxml',
 'openff_unconstrained-1.1.0.offxml',
 'openff_unconstrained-2.2.0.offxml',
 'openff_unconstrained-2.2.1.offxml',
 'openff-2.0.0-rc.1.offxml',
 'openff-1.3.0.offxml',
 'openff_unconstrained-1.2.0.offxml',
 'tip3p_fb-1.1.0.offxml',
 'openff_unc

In [9]:
from batter.ligand_process import LigandFactory

In [10]:
ligand_path = 'combine_top/g1i.sdf'
ind = 0
ligandff_folder = 'test_ligandff'
ligand_name = 'g1i'
retain_lig_prot = True
ligand_ff = 'gaff2'

In [11]:
ligand_factory = LigandFactory()
ligand = ligand_factory.create_ligand(
        ligand_file=ligand_path,
        index=ind,
        output_dir=ligandff_folder,
        ligand_name=ligand_name,
        retain_lig_prot=retain_lig_prot,
        ligand_ff=ligand_ff
)

In [12]:
ligand.prepare_ligand_parameters()

INFO | The net charge of the ligand g1i in combine_top/g1i.sdf is 1.0 
INFO | Ligand g1i parameters prepared: /oak/stanford/groups/rondror/users/yuzhuang/software/batter/batter/tests/ligand/test_ligandff/g1i.lib 


In [13]:
from openff.toolkit import ForceField, Molecule, Topology
from openfe import SmallMoleculeComponent

sage_ff14sb = ForceField("openff-2.2.0.offxml", "ff14sb_off_impropers_0.0.4.offxml")

In [14]:
topology = Topology()
topology.add_molecule(ligand.openff_molecule)

0

In [17]:
import openmm

In [18]:
system = sage_ff14sb.create_openmm_system(topology)

print(f"I see {system.getNumParticles()} atoms.")

for force in system.getForces():
    if isinstance(force, openmm.openmm.HarmonicBondForce):
        print(f"I see {force.getNumBonds()} bonds.")
    if isinstance(force, openmm.openmm.HarmonicAngleForce):
        print(f"I see {force.getNumAngles()} angles.")
    if isinstance(force, openmm.openmm.PeriodicTorsionForce):
        print(f"I see {force.getNumTorsions()} mixed proper and improper dihedrals.")

I see 31 atoms.
I see 112 mixed proper and improper dihedrals.
I see 57 angles.
I see 16 bonds.


In [21]:
parmed_structure = pmd.openmm.load_topology(topology.to_openmm(), system)

In [22]:
print(f"I see {len(parmed_structure.atoms)} atoms.")


print(f"I see {len(parmed_structure.bonds)} bonds.")
print(f"I see {len(parmed_structure.angles)} angles.")
print(f"I see {len(parmed_structure.dihedrals)} mixed proper and improper dihedrals.")

I see 31 atoms.
I see 32 bonds.
I see 57 angles.
I see 112 mixed proper and improper dihedrals.


In [23]:
parmed_structure.improper_types

TrackedList([
])

In [26]:
parmed_structure.save('test_ligand.pdb')

ValueError: Cannot write PDB file with no coordinates

In [ ]:
for old_atom, new_atom in zip(ligand_res.atoms, new_ligand_res.atoms):
    old_atom.name = new_atom.name  # optional
    old_atom.type = new_atom.type
    old_atom.atomic_number = new_atom.atomic_number
    old_atom.charge = new_atom.charge
    old_atom.mass = new_atom.mass
    old_atom.rmin = new_atom.rmin
    old_atom.epsilon = new_atom.epsilon
    old_atom.rmin14 = new_atom.rmin14
    old_atom.epsilon14 = new_atom.epsilon14

In [12]:
interchange = sage_ff14sb.create_interchange(topology)

In [15]:
interchange.to_prmtop(f"combine_top/g1i.prmtop")

In [39]:
from openff.interchange.drivers import get_amber_energies, get_openmm_energies

In [41]:
openmm_energies = get_openmm_energies(interchange)
print(openmm_energies.energies)

{'Bond': <Quantity(32.5903839, 'kilojoule / mole')>, 'Angle': <Quantity(87.4273526, 'kilojoule / mole')>, 'Torsion': <Quantity(26.8279295, 'kilojoule / mole')>, 'Nonbonded': <Quantity(218.72762, 'kilojoule / mole')>}


In [42]:
amber_energies = get_amber_energies(interchange)
print(amber_energies.energies)

UnsupportedExportError: vdW method no-cutoff not supported

In [20]:
import parmed as pmd

In [5]:
system = pmd.load_file("combine_top/full.prmtop", "combine_top/full.inpcrd")

In [16]:
ligand = pmd.load_file("combine_top/g1i.prmtop")

In [22]:
ligand_res = system.residues[285]

In [23]:
new_ligand_res = ligand.residues[0]

In [25]:
assert len(ligand_res.atoms) == len(new_ligand_res.atoms), "Mismatch in atom count"

In [ ]:
for old_atom, new_atom in zip(ligand_res.atoms, new_ligand_res.atoms):
    old_atom.name = new_atom.name  # optional
    old_atom.type = new_atom.type
    old_atom.atomic_number = new_atom.atomic_number
    old_atom.charge = new_atom.charge
    old_atom.mass = new_atom.mass
    old_atom.rmin = new_atom.rmin
    old_atom.epsilon = new_atom.epsilon
    old_atom.rmin14 = new_atom.rmin14
    old_atom.epsilon14 = new_atom.epsilon14